# Tutorial 6: More Informative Rendering

In this tutorial, we will discuss how to make rendering more informative. We will also demonstrate that rendering is agnostic to the selected physics-engine.

The following will be covered:
- Create an overlay node that augments a raw image sensors
- Connect the overlay node and use it for rendering
- Demonstrate that rendering is agnostic to the selected physics-engine

In the remainder of this tutorial, we will go more into detail on this concept.

Furthermore, at the end of this notebook you will find an exercise.
For the exercise you will have to add/modify a couple of lines of code, which are marked by

```python

# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

## Pendulum Swing-up

We will assume that we already have the object definition of the underactuated pendulum that we used in the [first](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/1_environment_creation.ipynb) tutorial with its dynamics simulated with the [OdeEngine](https://github.com/eager-dev/eagerx_ode). We will also assume that the *engine-specific* implementation we created in the [previous tutorial](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/5_engine_implementation.ipynb) for the [GymEngine](https://github.com/eager-dev/eagerx/blob/master/eagerx/engines/openai_gym/engine.py) is available.

Our goal is to make the rendered images more informative. We will lay the actions, selected by the agent, over the raw images produced by the image sensor of the pendulum. We will then visualise the augmented images instead of the raw images from the image sensor.

## Activate GPU (Colab only)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

In [1]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials  >> /tmp/eagerx_install.txt 2>&1"}
if 'google.colab' in str(get_ipython()):
  !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
  !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

Not running on CoLab.
Execute ROS commands as "!...".
ROS noetic available.


## Let's get started

We start by importing the required packages and initializing EAGERx.

In [2]:
import eagerx
import eagerx_tutorials.pendulum  # Registers Pendulum
import eagerx_tutorials.pendulum.gym_implementation  # Registers engine-specific implementation for the GymEngine  

# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core")

... logging to /home/jelle/.ros/log/fde154f4-d6a3-11ec-8700-5feb7550a31d/roslaunch-jelle-Alienware-m15-R4-132754.log
started roslaunch server http://145.94.219.234:34113/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

[INFO] [1652877820.255265]: Roscore cannot run as another roscore/master is already running. Continuing without re-initializing the roscore.


We will again create an environment with the *Pendulum* object, like we did in the [first](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/1_environment_creation.ipynb) and [second](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/2_reset_and_step.ipynb) tutorials.

Let's make the *Pendulum* object and add it to an empty graph.

In [3]:
pendulum = eagerx.Object.make("Pendulum", "pendulum", actuators=["u"], sensors=["theta", "dtheta", "image"], states=["model_state"])

# Define rate in Hz
rate = 30.0

# Initialize empty graph
graph = eagerx.Graph.create()

# Add pendulum to the graph
graph.add(pendulum)

# Connect the pendulum to an action and observation
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.dtheta, observation="angular_velocity")

Below we have defined a overlay node. The callback receives two inputs, namely the applied action `u` and a `raw_image`. In the callback, the applied action is visualised as an overlay on top of the raw image. The resulting `image` is the output of the callback. If we render this output, instead of the raw `pendulum.sensors.image`, we get a more informative rendered image.

In the exercise of this tutorial, we will finish the code that defines the Overlay node below. Specifically, we will overlay some text on top of the image that indicates the current timestamp.

Similar to what we covered in [this](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/4_nodes.ipynb) tutorial, we can create this node by inheriting from the class [`Node`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html).
This class has the following abstract methods we need to implement:

- [`spec()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.spec): Specifies the parameters of the node.
- [`initialize()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.initialize): Initializes the node.
- [`reset()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.reset): Resets the node at the beginning of an episode.
- [`callback()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.callback): Called at the rate of the node.


In [4]:
from eagerx.utils.utils import Msg
from std_msgs.msg import Float32MultiArray
from sensor_msgs.msg import Image
import cv2
import numpy as np


def convert_to_cv_image(img):
    if isinstance(img.data, bytes):
        cv_image = np.frombuffer(img.data, dtype=np.uint8).reshape(img.height, img.width, -1)
    else:
        cv_image = np.array(img.data, dtype=np.uint8).reshape(img.height, img.width, -1)
    if "rgb" in img.encoding:
        cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2BGR)
    return cv_image


class Overlay(eagerx.Node):
    @staticmethod
    @eagerx.register.spec("Overlay", eagerx.Node)
    def spec(
        spec: eagerx.specs.NodeSpec,
        name: str,
        rate: float,
        process: int = eagerx.process.ENVIRONMENT,
        color: str = "cyan",
    ):
        """Overlay spec"""
        # Fills spec with defaults parameters
        spec.initialize(Overlay)

        # Adjust default params
        spec.config.update(name=name, rate=rate, process=process, color=color, inputs=["raw_image", "u"], outputs=["image"])

    def initialize(self):
        """Nothing to initialize"""
        pass

    @eagerx.register.states()
    def reset(self):
        """Nothing to reset (i.e. stateless node)"""
        pass

    @eagerx.register.inputs(raw_image=Image, u=Float32MultiArray)
    @eagerx.register.outputs(image=Image)
    def callback(self, t_n: float, raw_image: Msg, u: Msg):
        """The callback where we overlay the additional info"""        
        # Convert action message to a float
        u = u.msgs[-1].data[0]

        # Set background image from raw_image
        img = convert_to_cv_image(raw_image.msgs[-1])
        
        # Get dimensions of the image
        width = raw_image.msgs[-1].width
        height = raw_image.msgs[-1].height
        height_bar = int(0.05 * height)
        width_bar = width - 2 * height_bar

        # Put text
        img = cv2.putText(img, "Applied Voltage", (179, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

        # Draw grey bar
        top_left = int(0.05 * height), int(0.05 * height)
        lower_right = int(width - height_bar), int(2 * 0.05 * height)
        img = cv2.rectangle(img, top_left, lower_right, (125, 125, 125), -1)

        # Fill bar proportional to the action that is applied
        top_left = int(width / 2), int(0.05 * height)
        lower_right = int(width / 2 + width_bar // 2 * u / 3),  int(2 * 0.05 * height)
        img = cv2.rectangle(img, top_left, lower_right, (0, 0, 0), -1)
        
        # Add text with timestamp to image (i.e. visualise t_n)
        # you can position it at (10, height - 20) for instance
        # (bottom left)
        # START EXERCISE 1.3
        
        # START EXERCISE 1.3
        
        # Prepare image for transmission.
        data = img.tobytes("C")
        msg = Image(data=data, height=height, width=width, encoding="bgr8", step=3 * width)
        return dict(image=msg)

Below, we wil initially render the raw images produced by the pendulum image sensor as we did in the preceding tutorials. In the exercise of this tutorial, we will:
- Add the overlay node to the graph.
- Connect the inputs of the overlay node. I.e. `u` to the `voltage` action and `pendulum.sensors.image` to the `overlay.inputs.raw_image`.
- Change the render source from `pendulum.sensors.image` to `overlay.outputs.raw_image`.

In [5]:
# Create the overlay node
overlay = eagerx.Node.make("Overlay", "overlay", rate)

# Copy the space converter of the actuator to the overlay node.
# This only suppresses a warning, that is otherwise harmless.
overlay.inputs.u.space_converter = pendulum.actuators.u.space_converter

# Add overlay node to graph and connect it
# START EXERCISE 1.1
# Add the overlay node to the graph and connect the inputs overlay.inputs.raw_image 
# and overlay.inputs.u to pendulum.sensors.image and action voltage, respectively.

# END EXERCISE 1.1

# Define the render source
# START EXERCISE 1.2
# Change the render source to overlay.outputs.image.
graph.render(source=pendulum.sensors.image, rate=rate)
# START EXERCISE 1.2

We will intially use the OdeEngine, as we did in the preceding tutorials. Later on in the exercises. we will switch and use the [GymEngine](https://github.com/eager-dev/eagerx/blob/master/eagerx/engines/openai_gym/engine.py) instead.

Because the overlay node is agnostic to the *engine-specific* implementation of the pendulum object, it will naturally overlay the additional information on whatever image it receives. Hence, this allows informative rendering to be available, whatever physics-engine is chosen.

In [6]:
# Register both engines
import eagerx_ode  # Registers OdeEngine
import eagerx.engines.openai_gym  # Registers GymEngine

# Make the engine
# START EXERCISE 1.4
engine = eagerx.Engine.make("OdeEngine", rate=rate)
# engine = eagerx.Engine.make("GymEngine", rate=rate, process=eagerx.process.ENVIRONMENT)
# graph.remove_component(pendulum.states.model_state)
# END EXERCISE 1.4

At this point, we have create a graph containing the pendulum. We provide the graph to the environment together with the engine. Based on this engine, we will initialize the *engine-specific implementation* for the pendulum that was registered with this engine. 
- If we use the [OdeEngine](https://github.com/eager-dev/eagerx_ode), the raw sensor images are produced by the custom render function in the registered OdeEngine implementation [here](https://github.com/eager-dev/eagerx_tutorials/blob/3ddc2eb7558c7825095611fec3a01a47f5e7af79/eagerx_tutorials/pendulum/objects.py#L108-L168).
- If we use the [GymEngine](https://github.com/eager-dev/eagerx/blob/master/eagerx/engines/openai_gym/engine.py), the raw sensor images are produced by the [Pendulum-v1](https://gym.openai.com/envs/Pendulum-v0/) environment.
- If we would have an implemention for the real-world and registered it with the [RealEngine](https://github.com/eager-dev/eagerx_reality/blob/m1aster/eagerx_reality/engine.py), the raw sensor images could, for example, be produced by a real camera.

Finally, we train the agent using [Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/), again similar to the preceding tutorials.

In [7]:
from typing import Dict
import numpy as np
import stable_baselines3 as sb3
from eagerx.wrappers import Flatten

# Define step function
def step_fn(prev_obs: Dict[str, np.ndarray], obs: Dict[str, np.ndarray], action: Dict[str, np.ndarray], steps: int):
    
    # Get angle and angular velocity
    # Take first element because of window size (covered in other tutorial)
    th = obs["angle"][0] 
        
    thdot = obs["angular_velocity"][0]
    
    # Convert from numpy array to float
    u = float(action["voltage"])
    
    # Calculate cost
    # Penalize angle error, angular velocity and input voltage
    cost = th**2 + 0.1 * thdot**2 + 0.001 * u**2  
    
    # Determine when is the episode over
    # currently just a timeout after 100 steps
    done = steps > 100
    
    # Set info, tell the algorithm the termination was due to a timeout
    # (the episode was truncated)
    info = {"TimeLimit.truncated": steps > 100}
    
    return obs, -cost, done, info

# Initialize Environment
env = eagerx.EagerxEnv(name="PendulumEnv", rate=rate, graph=graph, engine=engine, step_fn=step_fn)

# Toggle render
env.render("human")

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box()
env = Flatten(env)

# Initialize learner
model = sb3.SAC("MlpPolicy", env, verbose=1, device="cpu")

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(60 * rate))

env.shutdown()

[WARN] [1652877821.045856]: eagerx.EagerxEnv will be removed in the next release. Please subclass eagerx.BaseEnv instead.


[INFO] [1652877821.108543]: Node "/PendulumEnv/env/supervisor" initialized.
[INFO] [1652877821.253093]: Node "/PendulumEnv/engine" initialized.
[INFO] [1652877821.377069]: Node "/PendulumEnv/environment" initialized.
[INFO] [1652877821.471617]: Node "/PendulumEnv/pendulum/theta" initialized.
[INFO] [1652877821.517694]: Waiting for nodes "['env/render']" to be initialized.
[INFO] [1652877821.525162]: Node "/PendulumEnv/pendulum/dtheta" initialized.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[INFO] [1652877821.593940]: Adding object "pendulum" of type "Pendulum" to the simulator.
[INFO] [1652877821.614395]: Node "/PendulumEnv/pendulum/x" initialized.
[INFO] [1652877821.635387]: Node "/PendulumEnv/pendulum/image" initialized.
[INFO] [1652877821.635939]: [pendulum/image] START RENDERING!
[INFO] [1652877821.653010]: Node "/PendulumEnv/pendulum/u" initialized.
[INFO] [1652877821.666089]: Node "/PendulumEnv/pendulum/u_applied" initialized.
[I

# Exercise

In this exercise you will create a node that overlays the applied actions over raw images that are produced by the image sensor of the pendulum. As the overlay node is agnostic to the physics-engine, we have the same overlay in every physics-engine.

For this exercise, you will need to modify or add some lines of code in the cells above.
These lines are indicated by the following comments:

```python
# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

However, feel free to play with the other code as well if you are interested.
We recommend you to restart and run all code after each section (in Colab there is the option *Restart and run all* under *Runtime*).

## 1. Render more informative images


### Add your code to the following blocks: 

1.1 Add the overlay node to the graph and connect the inputs `overlay.inputs.raw_image` and `overlay.inputs.u` to `pendulum.sensors.image` and action `voltage`, respectively.  
1.2 Change the render source to `overlay.outputs.image`. Using the [*eagerx_gui* package](https://github.com/eager-dev/eagerx_gui), you would see that the graph looks as below if `graph.gui()` would be called. Run the code, and you should now see the rendered overlay instead of the raw sensor images. 

<img src="./figures/tutorial_6_gui.svg" width=720>

1.3 In the callback of the overlay node, add the current time (i.e. `t_n`) as text to the image. Run the code, and you should see a timestamp that increase while the episode progresses.  
1.4 Select the GymEngine by uncommenting the marked line. Also deselect the `model_state` by uncommenting the marked line of code.  Run the code, and you should see that the raw image has changed, but the overlay is still put on top. Hence, this demonstrates the agnostic behavior of the `graph`. 